# Evaluating Weekly Sales with Bottomup and Node predictions

In [3]:
import pandas as pd
import numpy as np

import pickle, time #, datetime, 



## Load and preprocess

In [49]:
t0 = time.time()

with open('/home/riag/scratch/data/parallel_leaves_rem.pickle', "rb") as f:
    pred = pickle.load(f)

pred = dict(pred) #if isinstance(pred, list)

print('Read non-leaves figures: %.1f seconds'%(time.time()-t0))


Read non-leaves figures: 10 seconds


In [50]:

len(pred),type(pred)

(3424, dict)

In [ ]:
pred.get(('category1',)).keys()

In [51]:
list(pred)[0]

('province1',)

In [52]:
# Create dataframes for true sales and predicted sales at any given node in the hierarchy except leaves
true = []
preds = []

for k,v, in pred.items():
    v['yhat'] = v['yhat'].rename(columns={'yhat':k})
    true.append(v['yhat'])
    
    v['y'] = v['y'].rename(columns={'y':k})
    preds.append(v['y'])
    
true_sales = pd.concat(true, axis=1).astype(float)
pred_sales = pd.concat(preds, axis=1).astype(float)


In [53]:
t0 = time.time()

with open("../data/agg_combo.pickle", "rb") as f:
    summation = pickle.load(f)

print('Read aggregation matrix: %.1f seconds'%(time.time()-t0))


Read aggregation matrix: 0 seconds


In [54]:
t0 = time.time()

# load the predicted value dictionary from file
with open( "/home/riag/scratch/data/parallel_leaves.pickle", "rb" ) as f:
    pred_leaves = pickle.load( f )
# add key as column name of series, return all series as dataframe

print('Read leaves models: %.1f seconds'%(time.time()-t0))


Read leaves models: 6 seconds


In [55]:
pred_leaves = dict(pred_leaves)
len(pred_leaves), type(pred_leaves)

(2217, dict)

In [56]:
pred_leaves.get(('store1', 'class1')).get('y').columns

Index(['y'], dtype='object')

In [57]:
t0 = time.time()

true_leaf = []
pred_leaf = []

for k,v, in pred_leaves.items():
    v['yhat'] = v['yhat'].rename(columns={'yhat':k})
    pred_leaf.append(v['yhat'])

    v['y'] = v['y'].rename(columns={'y':k})
    true_leaf.append(v['y'])

    
true_leaf = pd.concat(true_leaf, axis=1)
pred_leaf = pd.concat(pred_leaf, axis=1)

print('Completed restructuring predictions for purpose of aggregation in %.1f secs.'%(time.time() - t0))
true_leaf.head(5)


Completed restructuring predictions for purpose of aggregation in 2 secs.


,"(store1, class1)","(store1, class10)","(store1, class11)","(store1, class12)","(store1, class13)","(store1, class14)","(store1, class15)","(store1, class16)","(store1, class17)","(store1, class18)",...,"(store9, class38)","(store9, class39)","(store9, class4)","(store9, class40)","(store9, class41)","(store9, class5)","(store9, class6)","(store9, class7)","(store9, class8)","(store9, class9)"
ds,,,,,,,,,,,,,,,,,,,,,
2017-01-01,4508.407490,6357.995912,1949.153075,1300.911473,571.389689,116.563496,3029.222434,2483.402434,-424.971081,152.561047,...,509.965297,460.682936,7863.036351,695.666946,0.0,160.703350,503.537163,9787.191127,0.000000,246.411803
2017-01-08,3518.474855,6051.516767,1195.132957,1281.484224,945.649935,804.945224,2012.291636,1617.104242,0.000000,0.000000,...,-77.851845,334.977205,7955.172938,332.834494,0.0,0.000000,367.832112,5575.334886,0.000000,0.000000
2017-01-15,1920.655014,7820.610667,2898.231348,891.082219,1416.975004,454.969039,1673.314703,3398.625867,0.000000,254.268411,...,699.952368,142.847422,5850.459021,128.562680,0.0,342.833813,1301.340016,7881.177974,0.000000,0.000000
2017-01-22,7297.931949,9891.898288,5159.291769,551.391049,364.260926,1329.909500,5399.918251,6546.554509,1591.391706,152.561047,...,-41.425752,214.271133,8170.301156,137.133525,0.0,171.416907,2888.374875,6474.559408,396.401596,604.958833
2017-01-29,1710.526456,6310.999110,2435.477124,1661.815485,1449.901335,496.394792,1470.614211,7153.727477,0.000000,742.806595,...,128.562680,71.423711,6299.428469,0.000000,0.0,828.515048,2181.280136,7353.928139,0.000000,0.000000


## Bottom up aggregation

In [58]:

# Using the summation matrix to get predictions of every combination in the hierarchy


In [59]:
# Make predictions for upper levels using Bottom-up approach
t0 = time.time()

pred_leaf = pred_leaf.transpose() # (leaves x time) (2217x314)

# making the predictions, converting to numpy to enable multiplication

b = pred_leaf.to_numpy()
S_arr = summation.to_numpy()

ypred = np.dot(S_arr,b)

pred_sales_bu = pd.DataFrame(data=ypred, index=summation.index, 
                          columns=pred_leaf.columns.to_list())

print('Completed aggregation in %.1f secs.'%(time.time() - t0))
pred_sales_bu.head(5)

Completed aggregation in 30 secs.


,2017-01-01,2017-01-08,2017-01-15,2017-01-22,2017-01-29,2017-02-05,2017-02-12,2017-02-19,2017-02-26,2017-03-05,...,2017-11-05,2017-11-12,2017-11-19,2017-11-26,2017-12-03,2017-12-10,2017-12-17,2017-12-24,2017-12-31,2018-01-07
sales,2.56564e+06,2.10283e+06,1.78221e+06,1.77945e+06,1.95735e+06,2.03019e+06,1.87701e+06,1.65517e+06,1.6122e+06,1.82927e+06,...,2.79395e+06,3.116e+06,3.28424e+06,3.15036e+06,2.87871e+06,2.73859e+06,2.78442e+06,2.79614e+06,2.54873e+06,2.0946e+06
province1,664865,562520,503228,518777,564946,573811,529761,482523,488274,551299,...,759118,851966,898451,860125,782942,739143,743443,741141,679858,577653
city11,58097,46668.5,45158.8,53173.9,62813.2,66001.6,62250.7,58705.4,62481.3,73528.6,...,91529.2,90864.9,89388.7,87221.7,85508.3,84197.7,80494.1,71095.2,56834.2,44230.8
store47,25751.4,21470.5,22353.2,27528.2,32836.2,34338.5,32149.3,30479.3,33428.4,40738.4,...,47043.5,45124,42875,41336.2,40381.6,38892.9,35480,29583.1,22553.5,17551.4
category1,1.19809e+06,984953,838736,842629,931369,966841,888834,772879,741316,837677,...,1.24606e+06,1.39586e+06,1.4881e+06,1.44371e+06,1.32749e+06,1.26193e+06,1.2785e+06,1.28147e+06,1.16733e+06,958179


## Evaluate mse, mae, mape


### Actual vs Prediction at Node

In [ ]:
#defining metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

def mean_absolute_percentage_error2(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    zero = 1e-24 # to avoid division by zero
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true+zero))) * 100

MSE = mean_squared_error
MAE = mean_absolute_error
MAPE = mean_absolute_percentage_error2

S = pd.merge(true_sales, pred_sales, sort = False, left_index= True, right_index= True)
# S.columns.unique()

In [62]:
# Calculating metrics for for leaves

def remove_suffix(x):
    return x.split("_")[0]

def f(x):
    y_true = x.iloc[:,0].dropna()
    y_pred = x.iloc[:,1].dropna()
    
    metrics = {
    'mse' : mean_squared_error(y_true, y_pred),
    'mae' : mean_absolute_error(y_true, y_pred),
    'mape' : mean_absolute_percentage_error2(y_true, y_pred)
    }
    return pd.Series(metrics)
    
    
S = S.rename(columns = remove_suffix)
# print(S.columns)
M = S.groupby(by = S.columns, axis=1).apply(f)

M


,"('category1',)","('category2',)","('city10', 'category1')","('city10', 'category2')","('city10', 'class1')","('city10', 'class10')","('city10', 'class11')","('city10', 'class12')","('city10', 'class13')","('city10', 'class14')",...,"('store9', 'department11')","('store9', 'department12')","('store9', 'department2')","('store9', 'department3')","('store9', 'department4')","('store9', 'department5')","('store9', 'department7')","('store9', 'department8')","('store9', 'department9')","('store9',)"
mse,2.128297e+11,1.290219e+09,2.828103e+08,1.126882e+06,3.764550e+07,2.377268e+07,7.995768e+06,3.057817e+06,2.684534e+06,3.109445e+06,...,528626.396911,624189.382485,7.089721e+07,2.473457e+07,3.766654e+07,1.491412e+07,72489.747605,44052.328862,0.0,2.988105e+08
mae,3.594592e+05,2.725903e+04,1.397695e+04,8.721575e+02,5.892142e+03,4.202403e+03,1.907620e+03,1.480282e+03,1.122950e+03,1.494768e+03,...,513.343753,623.604241,6.588653e+03,3.886229e+03,4.215017e+03,3.396805e+03,249.293776,177.476359,0.0,1.341451e+04
mape,1.433279e+01,1.952484e+01,1.893505e+01,2.934573e+01,9.510607e+01,1.463405e+02,4.511559e+01,7.325869e+01,1.018139e+02,1.400899e+03,...,100.964300,31.931472,1.701272e+01,2.168590e+01,2.531950e+02,5.980667e+01,100.000000,79.603343,0.0,1.712705e+01


In [63]:
S['(\'store9\', \'department9\')'].head(5)

,"('store9', 'department9')","('store9', 'department9')"
ds,,
2017-01-01,0.0,0.0
2017-01-08,0.0,0.0
2017-01-15,0.0,0.0
2017-01-22,0.0,0.0
2017-01-29,0.0,0.0


### Actual vs Node vs Bottom up

In [ ]:
from ast import literal_eval as make_tuple

def parse(name):
#     print(name, type(name))
    return tuple(name.split('+'))
#     return make_tuple(name)

def prefix(row):
    if isinstance(row, str):
        return row.split("_")[0]
    return row
    
    
def evaluate(x):
    assert x.shape[1]>=2, x.columns
    
    y_true = x.iloc[:,0].dropna()
    y_pred = []
    num_cols = x.shape[1] - 1
    for col in range(0,num_cols):
        ser = x.iloc[:,col+1].dropna()
        y_pred.append(ser)
#     print(y_pred[0])
#     return
    metrics = {'mse':[], 'mae':[], 'mape':[]}
    for yhat in y_pred:   
        metrics['mse'].append(MSE(y_true, yhat))
        metrics['mae'].append(MAE(y_true, yhat))
        metrics['mape'].append(MAPE(y_true, yhat))
#     print(metrics)
    return pd.DataFrame.from_dict(metrics, orient='index', columns=['node', 'bottom up']) 


true_sales, pred_sales, pred_sales_bu
true_leaf, pred_leaf

true_sales_all = pd.concat([true_sales, true_leaf], axis=1)
pred_sales_all = pd.concat([pred_sales, pred_leaf.T], axis=1)

pred_sales_all.shape

bu = pred_sales_bu.iloc[1:].rename(index=parse)
bu = bu.transpose()
bu = bu.rename(columns=str)

S = pd.merge(true_sales_all, pred_sales_all, suffixes=('_true','_pred'), sort = False, left_index= True, right_index= True)
S.columns



In [65]:
K = pd.merge(S, bu, suffixes=('','_bu'), sort=False, left_index=True, right_index=True) 
K = K.rename(columns=prefix)
true_sales_all.shape, pred_sales_all.shape, bu.shape, K.shape

# M = K.groupby(by = K.columns, axis=1).apply(f2)


((54, 5641), (54, 5641), (54, 5641), (54, 16923))

In [67]:
M = K.groupby(by = K.columns, axis=1).apply(evaluate)


## Save results

In [139]:
M.to_csv("evaluation_metrics_weekly.csv")

## Exploring metrics...

In [ ]:
#M.loc['mape',:] < 10
J = M.copy()
for col in J.columns: 
    if 0.0 in J[col].values: 
        del J[col]


In [ ]:
J.min().min()
k = J.idxmin(axis = 1, skipna = True) 
#J[(('province2',), bottom up)]
J.idxmin(axis = 1, skipna = True) 

In [127]:
J["('province1', 'class42')","node"], J["('province1', 'class42')","bottom up"]

(mse      35.296153
 mae       4.024210
 mape    100.000000
 Name: (('province1', 'class42'), node), dtype: float64,
 mse     7.099264e+11
 mae     8.290587e+05
 mape    1.426696e+08
 Name: (('province1', 'class42'), bottom up), dtype: float64)

In [129]:
J["('province3', 'class42')","node"], J["('province3', 'class42')","bottom up"]

(mse      35.349805
 mae       3.842914
 mape    100.000000
 Name: (('province3', 'class42'), node), dtype: float64,
 mse     2.330679e+11
 mae     4.724023e+05
 mape    5.328765e+07
 Name: (('province3', 'class42'), bottom up), dtype: float64)

In [ ]:
J["('province2',)","bottom up"], J["('province2',)","node"]

In [ ]:
J["('category1',)","bottom up"], J["('category1',)","node"]

In [152]:
pd.read_csv("evaluation_metrics.csv", header=[0,1], index_col=0)


('category1',)               ('category2',)                \
               node     bottom up           node     bottom up   
mse    2.128297e+11  1.704708e+12   1.290219e+09  1.843563e+12   
mae    3.594592e+05  1.280186e+06   2.725903e+04  1.332577e+06   
mape   1.433279e+01  4.792974e+01   1.952484e+01  1.092595e+03   

     ('city10', 'category1')               ('city10', 'category2')  \
                        node     bottom up                    node   
mse             2.828103e+08  1.965909e+12            1.126882e+06   
mae             1.397695e+04  1.362952e+06            8.721575e+02   
mape            1.893505e+01  1.810273e+03            2.934573e+01   

                   ('city10', 'class1')                ...  \
         bottom up                 node     bottom up  ...   
mse   2.295643e+12         3.764550e+07  4.257458e+10  ...   
mae   1.488587e+06         5.892142e+03  2.028942e+05  ...   
mape  5.373004e+04         9.510607e+01  3.488520e+03  ...   

     ('store9', 'department5')               ('store9', 'department7')  \
                          node     bottom up                      node   
mse               1.491412e+07  1.276265e+10              72489.747605   
mae               3.396805e+03  1.079577e+05                249.293776   
mape              5.980667e+01  1.990221e+03                100.000000   

                   ('store9', 'department8')                \
         bottom up                      node     bottom up   
mse   1.711403e+10              44052.328862  1.006196e+11   
mae   1.235597e+05                177.476359  3.083903e+05   
mape  9.822734e+04                 79.603343  1.473875e+05   

     ('store9', 'department9')                 ('store9',)                
                          node     bottom up          node     bottom up  
mse                        0.0  2.585475e+10  2.988105e+08  1.649176e+08  
mae                        0.0  1.545018e+05  1.341451e+04  1.179498e+04  
mape                       0.0  1.545018e+31  1.712705e+01  1.454708e+01  

[3 rows x 11282 columns]

In [ ]:
import dash_table
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode() # Interactive plot (plotly)

go.Figure(layout={}, d)